<a href="https://colab.research.google.com/github/jacksonkarel/selfmodifai/blob/main/notebooks/replit_extension_gpt4_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 64.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 28.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 12.2 MB/s eta 0:00:00


In [2]:
from google.colab import files

uploaded = files.upload()

Saving keys.json to keys.json


In [3]:
import json
import openai

keys = json.load(open('keys.json'))
openai.api_key = keys['openai']

In [4]:
import os
os.environ["GIT_EMAIL"] = keys["git_email"]
os.environ["GIT_USERNAME"] = keys["git_username"]

In [ ]:
import os

!rm -rf /root/.ssh

!mkdir /root/.ssh

!tar xvzf ssh.tar.gz

!cp ssh-colab/* /root/.ssh && rm -rf ssh-colab && rm -rf ssh.tar.gz
!chmod 700 .shh

!touch /root/.ssh/known_hosts
!ssh-keyscan github.com >> /root/.ssh/known_hosts
!chmod 644 /root/.ssh/known_hosts

!git config --global user.email GIT_EMAIL
!git config --global user.name GIT_USERNAME
!git clone git@github.com:jacksonkarel/selfmodifai-alpaca-lora.git
os.chdir("selfmodifai-alpaca-lora")

In [5]:
access_token = keys["github_access_token"]

!git config --global user.email GIT_EMAIL
!git config --global user.name GIT_USERNAME

os.system(f"git clone https://{access_token}@github.com/jacksonkarel/selfmodifai-alpaca-lora.git")
os.chdir("selfmodifai-alpaca-lora")

In [ ]:
with open('messages.json') as json_file:
    messages = json.load(json_file)

messages.pop()

with open('messages.json', 'w') as outfile:
    json.dump(messages, outfile)

In [ ]:
!git restore messages.json

In [ ]:
!rm messages.json

In [20]:
import re
import json
from transformers import pipeline
from openai.error import InvalidRequestError

def update_messages(content, role, messages):
    new_message = {"role": role, "content": content}
    messages.append(new_message)

    print(f"Step: {content}")
    return messages

def format_nbl(non_bash_languages):
    if len(non_bash_languages) == 1:
        nbl_str = non_bash_languages[0].title()

    if len(non_bash_languages) == 2:
        first_lang = non_bash_languages[0]
        sec_lang = non_bash_languages[1]

        nbl_str = f"{first_lang} and {sec_lang}"

    else:
        nbl_str = ""

        for nbl in non_bash_languages[:-1]:
            nbl_str += f"{nbl}, "

        last_lang = non_bash_languages[-1]
        nbl_str += f"and {last_lang}"

    return nbl_str

def detect_non_bash_code(chatgpt_output):
    # Pattern to match code blocks
    pattern = r'```(\w+)\n(.*?)```'

    matches = re.findall(pattern, chatgpt_output, re.DOTALL)

    non_bash_languages = []

    for nb_match in matches:
        language = nb_match[0]

        # Check if the language is not bash
        if language.lower() != 'bash':
            non_bash_languages.append(language)

    return non_bash_languages


def handle_too_long_context(messages):
    print("too long context")
    messages = messages[:-1]
    full_context = "Condense the information from the following past conversation between us. Keep all of the information that is relevant to the task at hand and remove all that is not:\n"
    for message in messages[1:]:
        role = message["role"]
        content = message["content"]

        full_context += (f"{role}: {content}\n\n")

    print(full_context)
    system_turn = {'role': 'system', 'content': 'You are part of an agent that is modifying the code of a machine learning model. The agent is in the model directory. When you write bash commands they will be executed and the output will be sent back to you.'}

    less_messages = [system_turn, {'role': 'user', 'content': full_context}]

    try:
        response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=less_messages
                )
        less_messages = [system_turn, {'role': 'assistant', 'content': response["choices"][0]["message"]["content"]}]

    except InvalidRequestError as e:
            # Check if the error message matches the context length issue
            if "maximum context length" in str(e):
                 response, less_messages = handle_too_long_context(messages)

            else:
                # Re-raise the exception if it's not what we're looking for
                raise e

    return response, less_messages

def gpt4_agent():
    ls_stream = os.popen("ls")
    ls_output = ls_stream.read()
    ls_prompt = f"ls:\n{ls_output}"

    messages = [{"role": "system", "content": "You are part of an agent that is modifying the code of a machine learning model. The agent is in the model directory. When you write bash commands they will be executed and the output will be sent back to you."}, {"role": "user", "content": "I'm using you inside of an agent. My code is parsing your outputs and using it to execute bash commands. Modify the model architecture so that the model is more powerful."}, {"role": "user", "content": "Create bash commands that do that. Give me them one by one."}, {"role": "assistant", "content": "Sure! Please run the following bash command, and it will provide useful information about the model.\n\nTo ensure the model files are available in the current directory, run:\n```bash\nls\n```"}, {"role": "user", "content": ls_prompt}]

    while True:

        try:
            response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=messages
            )
        except InvalidRequestError as e:
            # Check if the error message matches the context length issue
            if "maximum context length" in str(e):
                response, messages = handle_too_long_context(messages)

                with open('messages.json', 'w') as outfile:
                    json.dump(messages, outfile)

            else:
                # Re-raise the exception if it's not what we're looking for
                raise e

        response_content = response["choices"][0]["message"]["content"]

        messages = update_messages(response_content, "assistant", messages)

        # Define the regular expression pattern
        pattern = r'```bash\n(.*?)\n```'

        bash_matches = re.findall(pattern, response_content, re.DOTALL)

        non_bash_languages = detect_non_bash_code(response_content)

        bash_response = "Create bash commands that do that. Give me them one by one."
        if bash_matches:
            content = ""
        # matches is now a list of all bash commands in the string
            for bash_command in bash_matches:
                content += f"{bash_command}:\n"
                stream = os.popen(bash_command)

                content += stream.read()


            if non_bash_languages:
                nbl_str = format_nbl(non_bash_languages)

                content += f"Those are the outputs from those bash commands. Can you write bash commands to implement the {nbl_str} code?"

            elif not content:
                content = "Ok, did that"

        elif non_bash_languages:
            languages = format_nbl(non_bash_languages)
            content = f"Write bash commands to implement those changes in the {languages} files."

        elif "?" not in response_content:
            content = bash_response

        else:
            classifier = pipeline("zero-shot-classification")
            labels = ["suggestion for what to do next", "inquisitive question", "asking somebody to do something"]
            results = classifier(sequences=response_content, candidate_labels=labels, hypothesis_template="This text is a {}")

            if results["labels"][0] == "suggestion for what to do next":
                content = bash_response

            else:
                content = "My goal is to make this machine learning model more powerful by improving its architecture. Find the answer to that question in that context. If you can't, try another step in improving the model."


        messages = update_messages(content, "user", messages)

In [21]:
gpt4_agent()

Step: Thanks for providing the list of files. However, it seems that the model files and architecture information are not immediately present in the given directory. To modify the model architecture, you'll need to locate and access the model's source code files, which are likely inside some of the directories listed here.

Please run the following command to search for Python files in the subdirectories:

```bash
find . -name "*.py"
```
Step: find . -name "*.py":
./utils/callbacks.py
./utils/prompter.py
./utils/__init__.py
./export_hf_checkpoint.py
./export_state_dict_checkpoint.py
./finetune.py
./generate.py

Step: The `finetune.py` file seems to be where the model is likely being defined and trained. To modify the model's architecture to make it more powerful, you may need to modify this file.

1. First, open the `finetune.py` file to view its contents:

```bash
cat finetune.py
```

2. Secondly, you can search for the lines of code in `finetune.py` where the model architecture is de

KeyboardInterrupt: ignored

In [15]:
!ls

alpaca_data_cleaned_archive.json  generate.py
alpaca_data_gpt4.json		  lengths.ipynb
alpaca_data.json		  LICENSE
DATA_LICENSE			  messages.json
docker-compose.yml		  pyproject.toml
Dockerfile			  README.md
export_hf_checkpoint.py		  requirements.txt
export_state_dict_checkpoint.py   templates
finetune.py			  utils
